## XGBoost

In [1]:
!pip install optuna 

                                              0.0/413.4 kB ? eta -:--:--
                                              0.0/413.4 kB ? eta -:--:--
                                              0.0/413.4 kB ? eta -:--:--
                                              0.0/413.4 kB ? eta -:--:--
                                              0.0/413.4 kB ? eta -:--:--
                                              0.0/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                    61.4/413.4 kB ? eta -:--:--
     -----                                  

In [74]:
# optuna : Hyper Parameter Optimization Framework
import optuna

In [75]:
!pip list

Package                       Version         Editable project location
----------------------------- --------------- -------------------------
aiobotocore                   2.4.2
aiofiles                      22.1.0
aiohttp                       3.8.3
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
alembic                       1.13.1
anaconda-catalogs             0.2.0
anaconda-client               1.11.3
anaconda-navigator            2.4.2
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat             

In [76]:
!pip install xgboost

In [77]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [78]:
data = pd.read_csv('Admission_Prediction.csv')

In [79]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Serial No.,500.0,250.500000,144.481833,1.00,125.7500,250.50,375.25,500.00
GRE Score,485.0,316.558763,11.274704,290.00,308.0000,317.00,325.00,340.00
TOEFL Score,490.0,107.187755,6.112899,92.00,103.0000,107.00,112.00,120.00
University Rating,485.0,3.121649,1.146160,1.00,2.0000,3.00,4.00,5.00
SOP,500.0,3.374000,0.991004,1.00,2.5000,3.50,4.00,5.00
LOR,500.0,3.484000,0.925450,1.00,3.0000,3.50,4.00,5.00
CGPA,500.0,8.576440,0.604813,6.80,8.1275,8.56,9.04,9.92
Research,500.0,0.560000,0.496884,0.00,0.0000,1.00,1.00,1.00
Chance of Admit,500.0,0.721740,0.141140,0.34,0.6300,0.72,0.82,0.97


In [80]:
data.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [81]:
data.shape

(500, 9)

In [82]:
data['GRE Score'] = data['GRE Score'].fillna(data['GRE Score'].median())
data['TOEFL Score'] = data['TOEFL Score'].fillna(data['TOEFL Score'].median())
data['University Rating'] = data['University Rating'].fillna(data['University Rating'].median())

In [83]:
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [84]:
X = data.drop(['Serial No.','Chance of Admit'], axis=1)

In [85]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [86]:
y = data['Chance of Admit']

In [87]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [88]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25, random_state=25)

In [89]:
X_train.shape

(375, 7)

In [90]:
y_test.shape

(125,)

In [91]:
sc = StandardScaler()

In [92]:
X_train_sc = sc.fit_transform(X_train)

In [93]:
X_test_sc= sc.fit_transform(X_test)

In [94]:
def objective(trail, data= X, target=y):
    train_X,test_X,train_y,test_y =train_test_split(data,target,test_size = 0.25, random_state=25)
    param = {
      'tree_method': 'hist',
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :3000,
      'max_depth' :trail.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state',[10,20,30,2000,3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight',1,200)
           
    }
    
    model =xgb.XGBRegressor(**param)
    model.fit(train_X,train_y,eval_set = [(test_X,test_y)], verbose =True)
    pred = model.predict(test_X)
    mse = mean_squared_error(test_y,pred)
    return mse

In [95]:
find_params = optuna.create_study()
find_params.optimize(objective,n_trials = 10)
find_params.best_trial.params

[I 2024-01-31 22:21:26,073] A new study created in memory with name: no-name-b5c0c24c-f661-4222-9290-41eb74554b5f


[0]	validation_0-rmse:0.14510
[1]	validation_0-rmse:0.14510
[2]	validation_0-rmse:0.14510
[3]	validation_0-rmse:0.14510
[4]	validation_0-rmse:0.14510
[5]	validation_0-rmse:0.14510
[6]	validation_0-rmse:0.14510
[7]	validation_0-rmse:0.14510
[8]	validation_0-rmse:0.14510
[9]	validation_0-rmse:0.14510
[10]	validation_0-rmse:0.14510
[11]	validation_0-rmse:0.14510
[12]	validation_0-rmse:0.14510
[13]	validation_0-rmse:0.14510
[14]	validation_0-rmse:0.14510
[15]	validation_0-rmse:0.14510
[16]	validation_0-rmse:0.14510
[17]	validation_0-rmse:0.14510
[18]	validation_0-rmse:0.14510
[19]	validation_0-rmse:0.14510
[20]	validation_0-rmse:0.14510
[21]	validation_0-rmse:0.14510
[22]	validation_0-rmse:0.14510
[23]	validation_0-rmse:0.14510
[24]	validation_0-rmse:0.14510
[25]	validation_0-rmse:0.14510
[26]	validation_0-rmse:0.14510
[27]	validation_0-rmse:0.14510
[28]	validation_0-rmse:0.14510
[29]	validation_0-rmse:0.14510
[30]	validation_0-rmse:0.14510
[31]	validation_0-rmse:0.14510
[32]	validation_0-

C:\Users\dell\AppData\Local\Temp\ipykernel_11604\3096090542.py:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

C:\Users\dell\AppData\Local\Temp\ipykernel_11604\3096090542.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[77]	validation_0-rmse:0.14510
[78]	validation_0-rmse:0.14510
[79]	validation_0-rmse:0.14510
[80]	validation_0-rmse:0.14510
[81]	validation_0-rmse:0.14510
[82]	validation_0-rmse:0.14510
[83]	validation_0-rmse:0.14510
[84]	validation_0-rmse:0.14510
[85]	validation_0-rmse:0.14510
[86]	validation_0-rmse:0.14510
[87]	validation_0-rmse:0.14510
[88]	validation_0-rmse:0.14510
[89]	validation_0-rmse:0.14510
[90]	validation_0-rmse:0.14510
[91]	validation_0-rmse:0.14510
[92]	validation_0-rmse:0.14510
[93]	validation_0-rmse:0.14510
[94]	validation_0-rmse:0.14510
[95]	validation_0-rmse:0.14510
[96]	validation_0-rmse:0.14510
[97]	validation_0-rmse:0.14510
[98]	validation_0-rmse:0.14510
[99]	validation_0-rmse:0.14510
[100]	validation_0-rmse:0.14510
[101]	validation_0-rmse:0.14510
[102]	validation_0-rmse:0.14510
[103]	validation_0-rmse:0.14510
[104]	validation_0-rmse:0.14510
[105]	validation_0-rmse:0.14510
[106]	validation_0-rmse:0.14510
[107]	validation_0-rmse:0.14510
[108]	validation_0-rmse:0.14510

[I 2024-01-31 22:21:32,509] Trial 0 finished with value: 0.02105384179126319 and parameters: {'lambda': 3.7352300583977947, 'alpha': 0.0001465993540489259, 'colsample_bytree': 0.1, 'subsample': 0.2, 'learning_rate': 0.01, 'max_depth': 9, 'random_state': 243123, 'min_child_weight': 124}. Best is trial 0 with value: 0.02105384179126319.


...

[55]	validation_0-rmse:0.14510
[56]	validation_0-rmse:0.14510
[57]	validation_0-rmse:0.14510
[58]	validation_0-rmse:0.14510
[59]	validation_0-rmse:0.14510
[60]	validation_0-rmse:0.14510
[61]	validation_0-rmse:0.14510
[62]	validation_0-rmse:0.14510
[63]	validation_0-rmse:0.14510
[64]	validation_0-rmse:0.14510
[65]	validation_0-rmse:0.14510
[66]	validation_0-rmse:0.14510
[67]	validation_0-rmse:0.14510
[68]	validation_0-rmse:0.14510
[69]	validation_0-rmse:0.14510
[70]	validation_0-rmse:0.14510
[71]	validation_0-rmse:0.14510
[72]	validation_0-rmse:0.14510
[73]	validation_0-rmse:0.14510
[74]	validation_0-rmse:0.14510
[75]	validation_0-rmse:0.14510
[76]	validation_0-rmse:0.14510
[77]	validation_0-rmse:0.14510
[78]	validation_0-rmse:0.14510
[79]	validation_0-rmse:0.14510
[80]	validation_0-rmse:0.14510
[81]	validation_0-rmse:0.14510
[82]	validation_0-rmse:0.14510
[83]	validation_0-rmse:0.14510
[84]	validation_0-rmse:0.14510
[85]	validation_0-rmse:0.14510
[86]	validation_0-rmse:0.14510
[87]	val

[I 2024-01-31 22:21:54,877] Trial 3 finished with value: 0.015268061102062112 and parameters: {'lambda': 0.20619979421451679, 'alpha': 0.003243553047384008, 'colsample_bytree': 0.8, 'subsample': 0.1, 'learning_rate': 0.02, 'max_depth': 3, 'random_state': 20, 'min_child_weight': 27}. Best is trial 1 with value: 0.003941096147984468.


[0]	validation_0-rmse:0.14510
[1]	validation_0-rmse:0.14510
[2]	validation_0-rmse:0.14510
[3]	validation_0-rmse:0.14510
[4]	validation_0-rmse:0.14510
[5]	validation_0-rmse:0.14510
[6]	validation_0-rmse:0.14510
[7]	validation_0-rmse:0.14510
[8]	validation_0-rmse:0.14510
[9]	validation_0-rmse:0.14510
[10]	validation_0-rmse:0.14510
[11]	validation_0-rmse:0.14510
[12]	validation_0-rmse:0.14510
[13]	validation_0-rmse:0.14510
[14]	validation_0-rmse:0.14510
[15]	validation_0-rmse:0.14510
[16]	validation_0-rmse:0.14510
[17]	validation_0-rmse:0.14510
[18]	validation_0-rmse:0.14510
[19]	validation_0-rmse:0.14510
[20]	validation_0-rmse:0.14510
[21]	validation_0-rmse:0.14510
[22]	validation_0-rmse:0.14510
[23]	validation_0-rmse:0.14510
[24]	validation_0-rmse:0.14510
[25]	validation_0-rmse:0.14510
[26]	validation_0-rmse:0.14510
[27]	validation_0-rmse:0.14510
[28]	validation_0-rmse:0.14510
[29]	validation_0-rmse:0.14510
[30]	validation_0-rmse:0.14510
[31]	validation_0-rmse:0.14510
[32]	validation_0-

C:\Users\dell\AppData\Local\Temp\ipykernel_11604\3096090542.py:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

C:\Users\dell\AppData\Local\Temp\ipykernel_11604\3096090542.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[38]	validation_0-rmse:0.14510
[39]	validation_0-rmse:0.14510
[40]	validation_0-rmse:0.14510
[41]	validation_0-rmse:0.14510
[42]	validation_0-rmse:0.14510
[43]	validation_0-rmse:0.14510
[44]	validation_0-rmse:0.14510
[45]	validation_0-rmse:0.14510
[46]	validation_0-rmse:0.14510
[47]	validation_0-rmse:0.14510
[48]	validation_0-rmse:0.14510
[49]	validation_0-rmse:0.14510
[50]	validation_0-rmse:0.14510
[51]	validation_0-rmse:0.14510
[52]	validation_0-rmse:0.14510
[53]	validation_0-rmse:0.14510
[54]	validation_0-rmse:0.14510
[55]	validation_0-rmse:0.14510
[56]	validation_0-rmse:0.14510
[57]	validation_0-rmse:0.14510
[58]	validation_0-rmse:0.14510
[59]	validation_0-rmse:0.14510
[60]	validation_0-rmse:0.14510
[61]	validation_0-rmse:0.14510
[62]	validation_0-rmse:0.14510
[63]	validation_0-rmse:0.14510
[64]	validation_0-rmse:0.14510
[65]	validation_0-rmse:0.14510
[66]	validation_0-rmse:0.14510
[67]	validation_0-rmse:0.14510
[68]	validation_0-rmse:0.14510
[69]	validation_0-rmse:0.14510
[70]	val

In [96]:
find_params.trails_dataframe().shape

In [97]:
optuna.visualization.plot_optimization_history(find_params)

In [98]:
optuna.visualization.plot_slice(find_params)

In [99]:
optuna.visualization.plot_contour(find_params,params=['alpha','lambda'])

In [100]:
best_params={'lambda': 0.7816338595163784,
 'alpha': 0.25683131748959687,
 'colsample_bytree': 0.1,
 'subsample': 0.9,
 'learning_rate': 1,
 'max_depth': 10,
 'random_state': 20,
 'min_child_weight': 124}

In [101]:
model=xgb.XGBRegressor(**best_params)

In [102]:
model.fit(X_train,y_train)

In [103]:
y_pred = model.predict(X_test)

In [104]:
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

In [105]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)